In [1]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mhcflurry import predict

Using TensorFlow backend.
/home/vasily/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
filepath = "./data/immunogenic_peptides.txt"
if os.path.isfile(filepath):
    df = pd.read_csv(filepath, sep="\t")
    print("Reading {}".format(os.path.basename(filepath)))
else:
    print("File path is wrong")

Reading immunogenic_peptides.txt


In [3]:
df.head(2)

,Peptide,MHC,Species,Immunogenicity
0,AAFDRKSDAK,HLA-A1101,Homo,immunogenic
1,AAFEFINSL,H-2-Kb,Mus,immunogenic


In [4]:
allelespath = "/home/vasily/.local/share/mhcflurry/4/0.0.8/models_class1_allele_specific_single/production.csv"
if os.path.isfile(allelespath):
    alleles_df = pd.read_csv(allelespath)
    print("Reading {}".format(os.path.basename(allelespath)))
else:
    print("File path is wrong")

Reading production.csv


In [5]:
alleles_df["allele"].head()

0    HLA-A0301
1    HLA-B3501
2     Mamu-B01
3    HLA-B4201
4    HLA-A0203
Name: allele, dtype: object

In [7]:
predictions = pd.DataFrame()
warning = pd.DataFrame()
warnings = 0
warning_loc = []
for i in range(len(df)):
    try:
        prediction = predict(alleles=df["MHC"].iloc[i:i+1].reset_index(drop=True),
                                   peptides=df["Peptide"].iloc[0:1].reset_index(drop=True))
        frame = [predictions, prediction]
        predictions = pd.concat(frame)
    except ValueError:
        warnings += 1
        frame = [warning, df.iloc[i]]
        warning = pd.concat(frame)
        warning_loc.append(i)
if len(predictions):
    print("Predictions made")
else:
    print("Something went wrong")
print("Number of warnings is {}".format(warnings))
print(warning)

Predictions made
Number of warnings is 1
                          0
Peptide           NIRQAGVQY
MHC               HLA-B1502
Species                Homo
Immunogenicity  immunogenic


In [29]:
"""
'HLA-B1502' allele causes an error in mhcflurry
"""
MHC_alleles = np.unique(df["MHC"])
idxs = np.where(MHC_alleles == 'HLA-B1502')
MHC_alleles = np.delete(MHC_alleles, idxs)
predict(alleles=MHC_alleles, peptides=df["Peptide"].iloc[0:1]).sort("Prediction", ascending=False)

/home/vasily/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,Allele,Peptide,Prediction
5,H-2-LD,AAFDRKSDAK,47724.140625
14,HLA-A2301,AAFDRKSDAK,38429.570312
34,HLA-B4501,AAFDRKSDAK,32892.187500
22,HLA-A6802,AAFDRKSDAK,32481.492188
31,HLA-B4002,AAFDRKSDAK,32466.308594
36,HLA-B5301,AAFDRKSDAK,32118.978516
15,HLA-A2402,AAFDRKSDAK,31825.773438
29,HLA-B3901,AAFDRKSDAK,31604.509766
0,H-2-DB,AAFDRKSDAK,30770.351562
16,HLA-A2601,AAFDRKSDAK,30111.015625
